In [1]:
import os
import sys
import cv2
import glob
import numpy as np
from time import time
from queue import Queue
from collections import namedtuple

sys.path.append('/home/huy/code/godofeye/lib')
sys.path.append('/home/huy/code/godofeye/lib/yoloface')

from blueeyes.face_recognition import FaceDetector, FaceRecognition, FeatureExtractor, ModelTraining
from blueeyes.utils import Camera

### Face Crop from Images (Optional)

In [ ]:
from pathlib import Path

IMAGES_DIR = '/home/huy/data/face_recog/train_test_raw/'
OUTPUT_DIR = '/home/huy/data/face_recog/train_test'

detector = FaceDetector('mtcnn', min_face_size=50)

count = 0

for img_path in glob.glob(IMAGES_DIR + '/**/*.jpg', recursive=True):
    path = Path(img_path)
    id = path.parent.name
    im = cv2.imread(str(path), 1)
    boxes = detector.detect(im)
    for left,top,right,bottom in boxes:
        crop = im[top:bottom,left:right,:]
        output_dir = OUTPUT_DIR + f'/{id}'
        output_path = output_dir + f'/{count}.jpg'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        cv2.imwrite(output_path, crop)
        print('Write to ', output_path)
        count += 1

### Create Train Test Set

In [2]:
from pathlib import Path

train_set_dict = {}
test_set_dict = {}

TRAINSET_LOCATION = '/home/huy/smartbuilding/face_recog_models/dataset/CBGVDataset_v2/*/WM/train/*.jpg'
TESTSET_LOCATION = '/home/huy/smartbuilding/face_recog_models/dataset/CBGVDataset_v2/*/WM/test/*.jpg'

for path in glob.glob(TRAINSET_LOCATION):
    path = Path(path)
    id = path.parent.parent.parent.name
    if id not in train_set_dict.keys():
        train_set_dict[id] = []
    train_set_dict[id].append(str(path)) 
for path in glob.glob(TESTSET_LOCATION):
    path = Path(path)
    id = path.parent.parent.parent.name
    if id not in test_set_dict.keys():
        test_set_dict[id] = []
    test_set_dict[id].append(str(path))

# for entry in os.scandir('/home/huy/face_recog/dataset/Data v4.1/train_set_mix'):
#     id = entry.name
#     train_paths = []
#     test_paths = []
#     all_paths = glob.glob(os.path.join(entry.path, '*'))
#     np.random.shuffle(all_paths)
#     for path in all_paths[2:len(all_paths)]:
#         train_paths.append(os.path.abspath(path))
#     for path in all_paths[0:2]:
#         test_paths.append(os.path.abspath(path))
# #     for path in all_paths:
# #         train_paths.append(os.path.abspath(path))
#     train_set_dict[id] = train_paths
#     test_set_dict[id] = test_paths

In [2]:
# auto split train test
from pathlib import Path

RATIO = 0.7

all_set_dict = {}
train_set_dict = {}
test_set_dict = {}

TRAINSET_LOCATION = '/home/huy/Downloads/StaffDATA_v1(CBGVDataset_v3.1)/Aug2/**/*.jpg'

for path in glob.glob(TRAINSET_LOCATION, recursive=True):
    path = Path(path)
    id = path.parent.name
    if id not in all_set_dict.keys():
        all_set_dict[id] = []
    all_set_dict[id].append(str(path)) 

for label, paths in all_set_dict.items():
    n = int(len(paths)*RATIO)
    train_set_dict[label] = paths[0:n]
    test_set_dict[label] = paths[n:]

In [ ]:
train_set_dict

### Train the model

In [4]:
feature_extractor = FeatureExtractor('face_recognition')
model_trainer = ModelTraining(feature_extractor=feature_extractor)

In [ ]:
model_trainer.create_train_set(train_set_dict, output_model_location='/home/huy/face_recog/encoded_data')

In [ ]:
model_trainer.create_train_set(test_set_dict, output_model_location='/home/huy/face_recog/encoded_data/test')

In [13]:
features = np.load('/home/huy/face_recog/encoded_data/train/features.dat')
labels = np.array(open('/home/huy/face_recog/encoded_data/train/labels.dat').readlines())

In [23]:
model_trainer.train_knn(features, labels, K=300, weights='uniform', output_model_location='/home/huy/face_recog/models/knn')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=300, p=2,
                     weights='uniform')


In [17]:
model = model_trainer.train_simple_model(features, labels, output_model_location='/home/huy/face_recog/models/simple_distance')

### Evaluate the model

In [24]:
recog = FaceRecognition(
    classifier_method='nn'
)
TP_count = 0
UNK_count = 0
num_samples = 0
for id, img_paths in test_set_dict.items():
    for path in img_paths:
        img = cv2.imread(path, 1)
        predict_id = recog.recog(img,[[0,0,img.shape[1],img.shape[0]]], threshold=0.9)
        predict_id = predict_id[0][0].split('\n')[0]
        if predict_id == id:
            TP_count += 1
        elif predict_id == 'unknown':
            UNK_count +=1
        num_samples += 1
# TP rate don't care UNK
print('num_samples\t', 'TP_count\t', 'UNK_count\t')
print(num_samples, TP_count, UNK_count)
print('TP Rate ',TP_count/(num_samples-UNK_count))
# False rate
# print(1 - (TP_count+UNK_count)/num_samples
print('UNK rate ', UNK_count/num_samples)

OSError: SavedModel file does not exist at: /home/huy/face_recog/models/nn/mobilenetv2_checkpoint_60-0.96.hdf5/{saved_model.pbtxt|saved_model.pb}

In [28]:
recog = FaceRecognition(
    model_dir='/home/huy/face_recog/models/knn/', 
   classifier_method='knn'
)
TP_count = 0
UNK_count = 0
num_samples = 0
for id, img_paths in test_set_dict.items():
    for path in img_paths:
        img = cv2.imread(path, 1)
        predict_id = recog.recog(img,[[0,0,img.shape[1],img.shape[0]]], threshold=0.5)
        predict_id = predict_id[0][0].split('\n')[0]
        if predict_id == id:
            TP_count += 1
        elif predict_id == 'unknown':
            UNK_count +=1
        num_samples += 1
# TP rate don't care UNK
print('num_samples\t', 'TP_count\t', 'UNK_count\t')
print(num_samples, TP_count, UNK_count)
print('TP Rate ',TP_count/(num_samples-UNK_count))
# False rate
# print(1 - (TP_count+UNK_count)/num_samples
print('UNK rate ', UNK_count/num_samples)

num_samples	 TP_count	 UNK_count	
28887 25379 3198
TP Rate  0.9879325781462883
UNK rate  0.1107072385502129


In [22]:
recog = FaceRecognition(
    model_dir='/home/huy/models/simple_distance/',
    feature_extractor_type='face_recognition'
)
TP_count = 0
UNK_count = 0
num_samples = 0
for id, img_paths in test_set_dict.items():
    for path in img_paths:
        img = cv2.imread(path, 1)
        predict_id = recog.recog(img,[[0,0,img.shape[1],img.shape[0]]], threshold=0.5)
        predict_id = predict_id[0][0].split('\n')[0]
        if predict_id == id:
            TP_count += 1
        elif predict_id == 'unknown':
            UNK_count +=1
        num_samples += 1
# TP rate don't care UNK
print('num_samples\t', 'TP_count\t', 'UNK_count\t')
print(num_samples, TP_count, UNK_count)
print('TP Rate ',TP_count/(num_samples-UNK_count))
# False rate
# print(1 - (TP_count+UNK_count)/num_samples
print('UNK rate ', UNK_count/num_samples)

num_samples	 TP_count	 UNK_count	
28887 10940 5016
TP Rate  0.4582966779774622
UNK rate  0.1736421227541801


#### KNN classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import pickle
import numpy as np

In [ ]:
KNN = pickle.load(open('/home/feature = [1]*128huy/Downloads/knn_clf.pkl', 'rb'))

In [ ]:
feature = np.random.random((128,1))

In [ ]:
x = KNN.predict_proba([feature])

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
KNN.algorithm